In [1]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players,teams
from nba_api.stats.endpoints import playbyplayv2
from nba_api.stats.endpoints import ShotChartDetail # For player shot location and frequency
from nba_api.stats.endpoints import commonplayerinfo # For Common Player Information
import requests
import os
import numpy as np
#from nba_api.stats.endpoints import playergamelogs
import pandas as pd
import time
def pull_rotation(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()

    frames = []
    for i in range(len(json["resultSets"])):
        data = json["resultSets"][i]["rowSet"]
        #print(data)
        columns = json["resultSets"][i]["headers"]
    #print(columns)

        df = pd.DataFrame.from_records(data, columns=columns)
        frames.append(df)
    gameframe = pd.concat(frames)
    time.sleep(1.5)
    
    return gameframe
def team_shotmap(team,season,stype ='Regular Season',league_id = '00',save_avg=False):
    carry = ''
    if stype!='Regular Season':
        carry ='ps'
   
    #print(team_id)
    nba_teams = teams.get_teams()
    team_list= {}
    full_name = {}
    for org in nba_teams:
        team_list[org['abbreviation']] = org['id']
        full_name[org['abbreviation']] = org['full_name']

    
    
    season_nullable= season
    team_id = team_list[team]
   
    shot_chart = ShotChartDetail(
                      player_id ='0',
                      
                      team_id=team_id,
                      season_nullable= season,
                      season_type_all_star=stype, 
                      context_measure_simple= 'FGA',
                                    league_id = league_id).get_data_frames()
        
    team_shots = shot_chart[0]
    if save_avg== True:
        
        avg = shot_chart[1]
        year = int(season.split('-')[0])+1
        avg.to_csv('team/'+str(year)+carry+'/avg.csv',index=False)
    return team_shots
def pull_rotation(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()

    frames = []
    for i in range(len(json["resultSets"])):
        data = json["resultSets"][i]["rowSet"]
        #print(data)
        columns = json["resultSets"][i]["headers"]
    #print(columns)

        df = pd.DataFrame.from_records(data, columns=columns)
        frames.append(df)
    gameframe = pd.concat(frames)
    return gameframe
def get_shotrotations(season,ps = False):
    stype = "Regular Season"
    carry = ""
    if ps == True:
        stype = "Playoffs"
        carry = "ps"
    nba_teams = teams.get_teams()
    
    team_list= {}
    full_name = {}
    for org in nba_teams:
        team_list[org['abbreviation']] = org['id']
        full_name[org['abbreviation']] = org['full_name']
    team_abr = full_name.keys()
    
    data = {}
    save_avg = True
    for team in team_abr:
        team = team.upper()
        
        shotmap = team_shotmap(team,season,stype=stype,save_avg=save_avg)
        save_avg=False
        games = list(set(shotmap.GAME_ID.tolist()))

        frames = []
        year =int(season.split('-')[0])+1
        path = 'rotations/'+str(year)+carry
        team_id=team_list[team]
        
        if os.path.exists(path+'/'+str(team_id)+'.csv'):
            
            temp = pd.read_csv(path+'/'+str(team_id)+'.csv')
            stored = set(temp.GAME_ID.tolist())
            frames.append(temp)
            new_games = []
            for game in games:
                if str(game)[0:2] !='00':
                    new_game = '00'+str(game)
                    new_games.append(new_game)
                else:
                    new_games.append(game)
                    
            games = new_games
            
            new_stored = []
            for game in stored:
                if str(game)[0:2] !='00':
                    new_game = '00'+str(game)
                    new_stored.append(new_game)
                else:
                    new_stored.append(game)
            stored=new_stored
  
            games = list(set(games) -set(stored))
            #print(games)
        for game in games:
            url='https://stats.nba.com/stats/gamerotation?GameID='+game+'&LeagueID=00'
            df = pull_rotation(url)

            frames.append(df)
        if len(shotmap)!=0:
            all_games = pd.concat(frames)
            all_games = all_games.drop_duplicates()
            all_games['GAME_ID']=all_games['GAME_ID'].astype(str)
            all_games['GAME_ID'] = np.where(all_games['GAME_ID'].str[0:2]=='00', all_games['GAME_ID'], '00'+all_games['GAME_ID'])
    
            all_games['GAME_ID'] = np.where(all_games['GAME_ID'].str[0:2]=='00', all_games['GAME_ID'], '00'+all_games['GAME_ID'])
    
    
            shotmap['GAME_ID'] = np.where(shotmap['GAME_ID'].str[0:2]=='00', shotmap['GAME_ID'], '00'+shotmap['GAME_ID'])
    
            
            to_merge = all_games[['GAME_ID','TEAM_ID','PERSON_ID','IN_TIME_REAL','OUT_TIME_REAL']].reset_index(drop=True)
            to_merge['GAME_ID']=to_merge['GAME_ID'].astype(str)
    
    
            #to_merge.rename(columns={'PERSON_ID':'PLAYER_ID'},inplace=True)
            to_merge = to_merge[to_merge.TEAM_ID==team_id]        
    
            shotmap['SHOT_ID'] = shotmap['GAME_ID'].astype(str)+shotmap['GAME_EVENT_ID'].astype(str)
            shotmap['PERIOD'].value_counts()
            shotmap['time'] = (shotmap['PERIOD']-1)*720+(12-(shotmap['MINUTES_REMAINING']+1))*60+(60-shotmap['SECONDS_REMAINING'])
            shotmap['extra'] = (shotmap['PERIOD']//5) *(shotmap['PERIOD']-4)
            shotmap['time'] = shotmap['time']-(shotmap['extra']*420)
            #shotmap[shotmap.extra>0]
            shotmap['time']*=10
            shot_times = shotmap.merge(to_merge,on=['GAME_ID','TEAM_ID'],how='left')
            shot_times = shot_times[shot_times.time>=shot_times.IN_TIME_REAL]
            shot_times = shot_times[shot_times.time<shot_times.OUT_TIME_REAL]
            shot_times.sort_values(by='GAME_DATE')
            shot_times['SHOT_ID'] = shot_times['SHOT_ID'].astype(str)
            shot_times['PERSON_ID'] =shot_times['PERSON_ID'].astype(str)
            shot_on = shot_times.groupby('SHOT_ID')['PERSON_ID'].apply('|'.join).reset_index()
            shot_on.columns=['SHOT_ID','PLAYERS_ON']
            #print(shot_on)
            print(len(shot_on))
            value_dict={}
            id_count = []
            for col in shot_on.PLAYERS_ON.tolist():
                count = len(col.split('|'))
                if count not in value_dict:
                    value_dict[count] = 1
                else:
                    value_dict[count]+=1
                id_count.append(count)
            shot_on['id_count']=id_count
            
            print(value_dict)
            if 'PLAYERS_ON' in shotmap.columns:
                shotmap = shotmap.drop(columns='PLAYERS_ON')
            final_shotmap = shotmap.merge(shot_on,how='left').reset_index(drop=True)
            #print(final_shotmap['PLAYERS_ON'])
            final_shotmap['PLAYERS_ON'] = np.where(final_shotmap['id_count']==5, final_shotmap['PLAYERS_ON'],final_shotmap['PLAYER_ID'].astype(str))
            #print(final_shotmap['PLAYERS_ON'])
            final_shotmap.PLAYERS_ON = final_shotmap.PLAYERS_ON.astype(str)
            value_dict={}
            for col in final_shotmap.PLAYERS_ON.tolist():
                count = len(col.split('|'))
                if count not in value_dict:
                    value_dict[count] = 1
                else:
                    value_dict[count]+=1
            print(value_dict)
            print(len(shotmap))
            year =int(season.split('-')[0])+1
            final_shotmap = final_shotmap.drop(columns='id_count')
            final_shotmap.to_csv('team/'+str(year)+carry+'/'+str(team_id)+'.csv',index = False)
            all_games = all_games[all_games.TEAM_ID==team_id]
            path = 'rotations/'+str(year)+carry
            isExist = os.path.exists(path)
            if not isExist:
    
       # Create a new directory because it does not exist
                os.makedirs(path)
                print('Making Folder ' +path)
            all_games.to_csv(path+'/'+str(team_id)+'.csv',index=False)
            
            data[team]=final_shotmap
        #print(team)
        #print(len(shotmap))
        #print(len(final_shotmap))
        #print(len(shot_on))
    return data
#data = get_shotrotations('2023-24',ps=False)
data = get_shotrotations('2024-25',ps=False)

6248
{5: 6248}
{5: 6248, 1: 8}
6256
6112
{5: 6112}
{5: 6112, 1: 11}
6123
6052
{5: 5981, 3: 33, 4: 34, 2: 3, 1: 1}
{5: 5981, 1: 76}
6057
6235
{5: 6235}
{5: 6235, 1: 5}
6240
6265
{5: 6265}
{5: 6265, 1: 4}
6269
6100
{5: 6100}
{5: 6100, 1: 8}
6108
6224
{5: 6224}
{5: 6224, 1: 2}
6226
6188
{5: 6188}
{5: 6188, 1: 2}
6190
6454
{5: 6400, 6: 32, 4: 22}
{5: 6400, 1: 56}
6456
5877
{5: 5798, 4: 14, 6: 11, 7: 27, 8: 16, 3: 11}
{5: 5798, 1: 82}
5880
5741
{5: 5741}
{5: 5741, 1: 2}
5743
5880
{5: 5880}
{5: 5880, 1: 5}
5885
5810
{5: 5810}
{5: 5810, 1: 7}
5817
6114
{5: 6020, 6: 12, 7: 8, 9: 17, 11: 27, 10: 12, 8: 18}
{5: 6020, 1: 102}
6122
5829
{5: 5829}
{5: 5829, 1: 12}
5841
5988
{5: 5988}
{5: 5988, 1: 6}
5994
5900
{5: 5900}
{5: 5900, 1: 6}
5906
5961
{5: 5801, 4: 56, 6: 31, 8: 2, 10: 3, 11: 8, 12: 48, 13: 4, 14: 2, 9: 5, 7: 1}
{5: 5801, 1: 160}
5961
5918
{5: 5918}
{5: 5918, 1: 2}
5920
5918
{5: 5918}
{5: 5918, 1: 3}
5921
6162
{5: 6162}
{5: 6162, 1: 10}
6172
6065
{5: 6065}
{5: 6065, 1: 4}
6069
6017
{5: 601

In [2]:
def assist_paths(ps = False):
    carry='ps'
    if ps == False:
        carry=''
    for year in range(1997,2026):
        path = 'assists/'+str(year)+carry
        isExist = os.path.exists(path)
        if not isExist:
        
        # Create a new directory because it does not exist
            os.makedirs(path)
            print('Making Folder ' +path)
def pull_assists(game_id):
    url='https://stats.nba.com/stats/playbyplayv2?EndPeriod=0&GameID='+game_id+'&StartPeriod=0'

    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    jsond = requests.get(url,headers = headers).json()

    data= jsond['resultSets'][0]['rowSet']
 


    columns = jsond['resultSets'][0]['headers']
    df = pd.DataFrame.from_records(data, columns=columns)
    
    
    df1 =df.dropna(subset=['HOMEDESCRIPTION'])
    df1=df1[(df1.HOMEDESCRIPTION.str.lower().str.contains('ast'))]
    
    df2 =df.dropna(subset=['VISITORDESCRIPTION'])
    
    df2 = df2[df2.VISITORDESCRIPTION.str.lower().str.contains('ast')]
    
    test_df = pd.concat([df1,df2])

    test_df['PLAYER1_TEAM_ID']= test_df['PLAYER1_TEAM_ID'].astype(int)
  
    col = ['GAME_ID', 'EVENTNUM','PLAYER1_ID','PLAYER2_ID', 'PLAYER1_TEAM_ID']
    test_df= test_df[col]
    for c in col:
        test_df[col] = test_df[col].astype(str)
    test_df.columns =  ['GAME_ID', 'EVENTNUM','PLAYER_ID','ASSIST_ID', 'TEAM_ID']
    test_df['SHOT_ID'] = test_df['GAME_ID']+test_df['EVENTNUM']
    return test_df

def list_csv_files(path):
    csv_files = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            csv_files.append(file)
    return csv_files
def scrape_assists(ps = False):
    carry="ps"
    if ps == False:
        carry=""
    print('Starting')
    for year in range (2025,2026):
        print(year)
        path = "rotations/"+str(year)+carry
        
        files = list_csv_files(path)
   
        files = [f for f in files if 'vs' not in f]
        files = [f for f in files if 'avg' not in f]
     
        year_data = []
        teams=[]
        games = []
        for f in files:
            teamdf = pd.read_csv(path+'/'+f)
            teamdf['GAME_ID'] = teamdf['GAME_ID'].astype(str)
            games = games+ list(teamdf['GAME_ID'].unique())
            del teamdf
        games = list(set(games))


    
        assist_path = 'assists/'+str(year)+carry+'/ast.csv'
        if os.path.exists(assist_path):
     
            logged = pd.read_csv(assist_path)
            logged['GAME_ID'] = logged['GAME_ID'].astype(str)
            logged_games = list(logged['GAME_ID'].unique())
            print(len(logged_games))
            games = [game for game in games if game not in logged_games]
            
        print(len(games))
        
        game_count = 0
        if game_count % 25 == 0:
            print('test')
           
        for game in games:
            game = str(game)
            if game[0:2] !='00':
                game = '00'+game
            df = pull_assists(game)
       
            year_data.append(df)
            game_count+=1

            if game_count %50 == 0:
                if os.path.exists(assist_path):
                    old_frame = pd.read_csv(assist_path)
                    new_frame = pd.concat(year_data)
    
                    to_save = pd.concat([old_frame,new_frame])
                    to_save.to_csv(assist_path,index = False)
                    year_data = []
    
                    print('50 hit, saving')
                else:
                    to_save = pd.concat(year_data)
                    to_save.to_csv(assist_path)
                    year_data= []
                    print('50 hit, saving')
        if len(games)  >0:          
            old_frame = pd.read_csv(assist_path)
            new_frame = pd.concat(year_data)
    
            to_save = pd.concat([old_frame,new_frame])
            to_save.to_csv(assist_path,index = False)
    
            print(' Year done ' +str(year)+carry)
        else:
            print('already scraped')
            print('Hit')
#assist_paths(ps=False)
#assist_paths(ps=True)
scrape_assists(ps=False)
def clean_assists(ps = False):
    carry="ps"
    if ps == False:
        carry=""

    for year in range (1997,2026):
  
        path = "assists/"+str(year)+carry+"/ast.csv"
        df = pd.read_csv(path)
        if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'],inplace=True)
        df.to_csv(path,index=False)
clean_assists()


Starting
2025
1022
0
test
already scraped
Hit


In [16]:
ps = ''
master = []
year = 2025

# Load year assists
year_assists = pd.read_csv(f'assists/{year}/ast.csv')

# Ensure SHOT_ID has '00' at the front
year_assists['SHOT_ID'] = year_assists['SHOT_ID'].astype(str)


# Load all shots data
for team in teams.get_teams():
    team_id = team['id']
    file_path = f'team/{year}{ps}/{team_id}.csv'
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        master.append(df)
    else:
        print(f"File not found: {file_path}")

# Combine all shots into a single dataframe
all_shots = pd.concat(master, ignore_index=True)

# Ensure SHOT_ID in all_shots is string for comparison
all_shots['SHOT_ID'] = all_shots['SHOT_ID'].astype(str)

# Create 'assisted' column: 1 if assisted, 0 otherwise
all_shots['assisted'] = all_shots['SHOT_ID'].isin(year_assists['SHOT_ID']).astype(int)

# Print the result to verify
print(all_shots.head())


print(all_shots['assisted'].sum()/len(all_shots))

           GRID_TYPE   GAME_ID  GAME_EVENT_ID  PLAYER_ID         PLAYER_NAME  \
0  Shot Chart Detail  22400001              7    1642258  Zaccharie Risacher   
1  Shot Chart Detail  22400001             10    1630552       Jalen Johnson   
2  Shot Chart Detail  22400001             21    1630552       Jalen Johnson   
3  Shot Chart Detail  22400001             34    1630811      Keaton Wallace   
4  Shot Chart Detail  22400001             36     203991        Clint Capela   

      TEAM_ID      TEAM_NAME  PERIOD  MINUTES_REMAINING  SECONDS_REMAINING  \
0  1610612737  Atlanta Hawks       1                 11                 43   
1  1610612737  Atlanta Hawks       1                 11                 38   
2  1610612737  Atlanta Hawks       1                 10                 50   
3  1610612737  Atlanta Hawks       1                  9                 47   
4  1610612737  Atlanta Hawks       1                  9                 44   

   ... SHOT_ATTEMPTED_FLAG SHOT_MADE_FLAG GAME_DAT

In [4]:

for team in teams.get_teams():
    
    team_id=team['id']
    team_games = all_shots[all_shots.TEAM_ID==team['id']].GAME_ID.unique()
    team_games = list(team_games)
    opp_shots = all_shots[(all_shots.TEAM_ID!=team['id']) &(all_shots.GAME_ID.isin(team_games))].reset_index(drop=True)

    opp_shots['PLAYER_ID']='0'
    opp_shots['PLAYER_NAME']='Opponent'
    opp_shots['TEAM_ID']=team_id
    opp_shots.drop(columns=['PLAYERS_ON'],inplace=True)
    shotmap = opp_shots
                                        
    shotmap['GAME_ID'] = shotmap['GAME_ID'].astype(str)
    path = 'rotations/'+str(year)+ps

    if os.path.exists(path+'/'+str(team_id)+'.csv'):
        print(team)
        all_games = pd.read_csv(path+'/'+str(team_id)+'.csv')
        all_games = all_games.drop_duplicates()
        all_games = all_games[all_games.TEAM_ID==team_id]
    
        #print(all_games)
        
        all_games['GAME_ID']=all_games['GAME_ID'].astype(str)
        all_games['GAME_ID'] = np.where(all_games['GAME_ID'].str[0:2]=='00', all_games['GAME_ID'], '00'+all_games['GAME_ID'])
    
     
        shotmap['GAME_ID'] = np.where(shotmap['GAME_ID'].str[0:2]=='00', shotmap['GAME_ID'], '00'+shotmap['GAME_ID'])
    
        #print(len(shotmap))
        to_merge = all_games[['GAME_ID','TEAM_ID','PERSON_ID','IN_TIME_REAL','OUT_TIME_REAL']].reset_index(drop=True)
        to_merge['GAME_ID']=to_merge['GAME_ID'].astype(str)
    
    
        #to_merge.rename(columns={'PERSON_ID':'PLAYER_ID'},inplace=True)
        to_merge = to_merge[to_merge.TEAM_ID==team_id]        
        
        shot_times = shotmap.merge(to_merge,on=['GAME_ID','TEAM_ID'])
        #print(len(shot_times))
        shot_times = shot_times[shot_times.time>=shot_times.IN_TIME_REAL]
        shot_times = shot_times[shot_times.time<shot_times.OUT_TIME_REAL]
        #print(len(shot_times))
        shot_times.sort_values(by='GAME_DATE')
        shot_times['SHOT_ID'] = shot_times['SHOT_ID'].astype(str)
        shot_times['PERSON_ID'] =shot_times['PERSON_ID'].astype(str)
        shot_on = shot_times.groupby('SHOT_ID')['PERSON_ID'].apply('|'.join).reset_index()
        shot_on['SHOT_ID'] = shot_on['SHOT_ID'].astype(str)
        shotmap['SHOT_ID'] = shotmap['SHOT_ID'].astype(str)
        shot_on.columns=['SHOT_ID','PLAYERS_ON']
        #print(len(shot_on))
        value_dict = {}
        id_count=[]
        for col in shot_on.PLAYERS_ON.tolist():
            count = len(col.split('|'))
            if count not in value_dict:
                value_dict[count] = 1
            else:
                value_dict[count]+=1
            id_count.append(count)
        print(value_dict)
        shot_on['id_count'] = id_count
        final_shotmap = shotmap.merge(shot_on,how='left')
        final_shotmap['PLAYERS_ON'] = np.where(final_shotmap['id_count']==5, final_shotmap['PLAYERS_ON'],final_shotmap['PLAYER_ID'].astype(str))
        value_dict = {}
        id_count=[]
        for col in final_shotmap.PLAYERS_ON.tolist():
            count = len(col.split('|'))
            if count not in value_dict:
                value_dict[count] = 1
            else:
                value_dict[count]+=1
              
            id_count.append(count)
        print(value_dict)
        final_shotmap = final_shotmap.drop(columns='id_count')
        final_shotmap.to_csv('team/'+str(year)+ps+'/'+str(team_id)+'vs.csv',index = False)
        print(final_shotmap['TEAM_NAME'].unique())
        #print(final_shotmap.head())


{'id': 1610612737, 'full_name': 'Atlanta Hawks', 'abbreviation': 'ATL', 'nickname': 'Hawks', 'city': 'Atlanta', 'state': 'Georgia', 'year_founded': 1949}
{5: 6163}
{5: 6163, 1: 3}
['Boston Celtics' 'Cleveland Cavaliers' 'New Orleans Pelicans'
 'Chicago Bulls' 'Dallas Mavericks' 'Denver Nuggets'
 'Golden State Warriors' 'Houston Rockets' 'LA Clippers'
 'Los Angeles Lakers' 'Miami Heat' 'Milwaukee Bucks'
 'Minnesota Timberwolves' 'Brooklyn Nets' 'New York Knicks'
 'Orlando Magic' 'Indiana Pacers' 'Philadelphia 76ers' 'Phoenix Suns'
 'Portland Trail Blazers' 'Sacramento Kings' 'San Antonio Spurs'
 'Oklahoma City Thunder' 'Toronto Raptors' 'Utah Jazz' 'Memphis Grizzlies'
 'Washington Wizards' 'Detroit Pistons' 'Charlotte Hornets']
{'id': 1610612738, 'full_name': 'Boston Celtics', 'abbreviation': 'BOS', 'nickname': 'Celtics', 'city': 'Boston', 'state': 'Massachusetts', 'year_founded': 1946}
{5: 6082}
{5: 6082, 1: 5}
['Atlanta Hawks' 'Cleveland Cavaliers' 'New Orleans Pelicans'
 'Chicago Bul

In [5]:
df.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM', 'SHOT_ID', 'time', 'extra', 'PLAYERS_ON'],
      dtype='object')

In [6]:
players = all_shots.PLAYER_ID.unique().tolist()
for player in players:
    df = all_shots[all_shots.PLAYER_ID==player]
    columns = ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE','assisted',
       'HTM', 'VTM']
    df=df[columns]
    df.to_csv('2025/'+str(player)+'.csv',index=False)


In [7]:
def get_rotations(season,ps=False):
    nba_teams = teams.get_teams()
    team_list= {}
    full_name = {}
    for org in nba_teams:
        team_list[org['abbreviation']] = org['id']
        full_name[org['abbreviation']] = org['full_name']
    team_abr = full_name.keys()
    
    data = []
    save_avg = True
    stored=[]
    missed_games = []
    carry=''
    if ps==True:
        carry='ps'
    for team in team_abr:
        team = team.upper()
        team_id=team_list[team]
        year =int(season.split('-')[0])+1
        
        shotmap_url='https://raw.githubusercontent.com/gabriel1200/shot_data/master/team/'+str(year)+carry+'/'+str(team_id)+'.csv'
        
        response = requests.get(shotmap_url)
        if response.status_code != 404:
            print(shotmap_url)
            shotmap = pd.read_csv(shotmap_url)
            save_avg=False
            games = list(set(shotmap.GAME_ID.tolist()))

            frames = []

            path = 'rotations/'+str(year)
            team_id=team_list[team]


            new_games = []
            for game in games:
                game = str(game)
                if game[0:2]!='00':
                    game = '00'+game
                    new_games.append(game)
                else:
                    new_games.append(games)
            games = new_games



            games = list(set(games) -set(stored))
            print(len(games))

            for game in games:
                game = str(game)
                if game[0:2]!='00':
                    game = '00'+game

                url='https://stats.nba.com/stats/gamerotation?GameID='+game+'&LeagueID=00'
                try:
                    df = pull_rotation(url)

                    frames.append(df)
                except ValueError:
                    print(game)
                    missed_games.append(game)
            
            if len(frames)!=0:
                all_games = pd.concat(frames)

                games =all_games['GAME_ID'].unique().tolist()
                stored =stored+games


                data.append(all_games)

        print(team)
            #print(len(shotmap))
            #print(len(final_shotmap))
            #print(len(shot_on))

    print(missed_games)
    return pd.concat(data)


In [8]:
'''
seasons = [str(year)+'-'+str(year+1)[-2:] for year in range (1997,1999)]

for season in seasons:
    year = int(season.split('-')[0])+1
    print(season)
    data = get_rotations(season)
   
    data.to_csv('rotations/'+str(year)+'.csv',index=False)
    
seasons = [str(year)+'-'+str(year+1)[-2:] for year in range (1996,2001)]

for season in seasons:
    year = int(season.split('-')[0])+1
    print(season)
    data = get_rotations(season,ps=True)
   
    data.to_csv('rotations/'+str(year)+'ps.csv',index=False)

for year in range(1997,2001):
        path = 'rotations/'+str(year)
        
        isExist = os.path.exists(path)
        if not isExist:

   # Create a new directory because it does not exist
            os.makedirs(path)
            print('Making Folder ' +path)
        df =pd.read_csv(path+'.csv')
        teams = df.TEAM_ID.unique().tolist()
        
        for team in teams:
            teamdf=df[df.TEAM_ID==team]
            
            teamdf.to_csv(path+'/'+str(team)+'.csv',index=False)
        print(year)

for year in range(1997,2001):
        path = 'rotations/'+str(year)+'ps'
        
        isExist = os.path.exists(path)
        if not isExist:

   # Create a new directory because it does not exist
            os.makedirs(path)
            print('Making Folder ' +path)
        df =pd.read_csv(path+'.csv')
        teams = df.TEAM_ID.unique().tolist()
        
        for team in teams:
            teamdf=df[df.TEAM_ID==team]
            
            teamdf.to_csv(path+'/'+str(team)+'.csv',index=False)
'''

"\nseasons = [str(year)+'-'+str(year+1)[-2:] for year in range (1997,1999)]\n\nfor season in seasons:\n    year = int(season.split('-')[0])+1\n    print(season)\n    data = get_rotations(season)\n   \n    data.to_csv('rotations/'+str(year)+'.csv',index=False)\n    \nseasons = [str(year)+'-'+str(year+1)[-2:] for year in range (1996,2001)]\n\nfor season in seasons:\n    year = int(season.split('-')[0])+1\n    print(season)\n    data = get_rotations(season,ps=True)\n   \n    data.to_csv('rotations/'+str(year)+'ps.csv',index=False)\n\nfor year in range(1997,2001):\n        path = 'rotations/'+str(year)\n        \n        isExist = os.path.exists(path)\n        if not isExist:\n\n   # Create a new directory because it does not exist\n            os.makedirs(path)\n            print('Making Folder ' +path)\n        df =pd.read_csv(path+'.csv')\n        teams = df.TEAM_ID.unique().tolist()\n        \n        for team in teams:\n            teamdf=df[df.TEAM_ID==team]\n            \n          

In [9]:

start_year=1997
end_year=2026

def get_dates(start_year,end_year):
    dates=[]
    for year in range(start_year,end_year):
    
        for team in teams.get_teams():
            team_id=team['id']
            path = 'team/'+str(year)+'ps/'+str(team_id)+'.csv'
            if os.path.exists(path):
                df=pd.read_csv(path)
    
                df=df[['GAME_ID','TEAM_ID','HTM','VTM','GAME_DATE']]
                df.rename(columns={'GAME_DATE':'date'},inplace=True)
                df.drop_duplicates(inplace=True)
                df['season']=str(year-1)+'-'+str(year)[-2:]
                df['playoffs']=True
                dates.append(df)
        for team in teams.get_teams():
            team_id=team['id']
            path = 'team/'+str(year)+'/'+str(team_id)+'.csv'
            if os.path.exists(path):
                df=pd.read_csv(path)
    
                df=df[['GAME_ID','TEAM_ID','HTM','VTM','GAME_DATE']]
                df.rename(columns={'GAME_DATE':'date'},inplace=True)
                df.drop_duplicates(inplace=True)
                df['season']=str(year-1)+'-'+str(year)[-2:]
                df['playoffs']=False
                dates.append(df)
                
    return pd.concat(dates)


dates=get_dates(start_year,end_year)
dates[dates.playoffs==False]
name_map={}
for team in teams.get_teams():

    name_map[team['id']]= team['abbreviation']

dates['team']=dates['TEAM_ID'].map(name_map)
acronym_changes = {
    "CHH": "CHA",
    "NOH": "NOP",
    "NJN": "BKN",
    "SEA": "OKC",
    "WSB": "WAS",
    "VAN": "MEM",
    "SDC": "LAC",
    "KCK": "SAC",
    "FTW": "DET",
    "SFW": "GSW",
    "STL": "ATL"
}
dates['team'] = dates['team'].replace(acronym_changes)

dates['HTM'] = dates['HTM'].replace(acronym_changes)
dates['VTM'] = dates['VTM'].replace(acronym_changes)

dates['opp_team'] = dates.apply(lambda row: row['VTM'] if row['team'] == row['HTM'] else row['HTM'], axis=1)

dates.sort_values(by='date',inplace=True)
dates.to_csv('game_dates.csv',index=False)
dates.to_csv('../web_app/data/game_dates.csv',index=False)

In [10]:
dates[dates.playoffs==False].tail(20)

,GAME_ID,TEAM_ID,HTM,VTM,date,season,playoffs,team,opp_team
5692,22400986,1610612754,MIN,IND,20250317,2024-25,False,IND,MIN
5986,22400989,1610612741,UTA,CHI,20250317,2024-25,False,CHI,UTA
5955,22400990,1610612743,GSW,DEN,20250317,2024-25,False,DEN,GSW
5787,22400992,1610612758,SAC,MEM,20250317,2024-25,False,SAC,MEM
6170,22400985,1610612745,HOU,PHI,20250317,2024-25,False,HOU,PHI
5774,22400995,1610612764,POR,WAS,20250317,2024-25,False,WAS,POR
5768,22400989,1610612762,UTA,CHI,20250317,2024-25,False,UTA,CHI
5941,22400991,1610612761,PHX,TOR,20250317,2024-25,False,TOR,PHX
3110,22400537,1610612759,LAL,SAS,20250317,2024-25,False,SAS,LAL
6169,22400992,1610612763,SAC,MEM,20250317,2024-25,False,MEM,SAC


In [11]:
width = 2400
height = 2000

# Calculate aspect ratios
aspect_ratio_1 = width / height
aspect_ratio_2 = width / height

# Calculate combined aspect ratio
aspect_ratio_combined = aspect_ratio_1 + aspect_ratio_2

# Set the dimensions of the combined image
combined_width = 2 * width  # Assuming you are combining two images side by side
combined_height = int(combined_width / aspect_ratio_combined)

# Print the dimensions of the combined image
print(f"Combined Image Dimensions: {combined_width} x {combined_height}")

Combined Image Dimensions: 4800 x 2000
